In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
import AC_VC_scripts.figure_formatting as ff
warnings.filterwarnings('ignore')
import AC_VC_scripts.loading_functions as lf
import AC_VC_scripts.MAPseq_data_processing as mdp
import AC_VC_scripts.fig2_plotting as pltt
from AC_VC_scripts import MAPseq_flatmap_functions as mff
%matplotlib inline

Load filepaths and perform area assignment

In [2]:
gen_parameters = lf.load_gen_params()
mice = gen_parameters['MICE']
saving_path = gen_parameters['fig_saving_path']
proj_path = gen_parameters['proj_path']
ff.set_font_params(gen_parameters)


In [3]:
#load all mice neuron projection datasets
combined_dict = mdp.samples_to_areas(mice = mice, proj_path=proj_path)
all_mice_combined = pd.concat([
    combined_dict[k]['homogenous_across_cubelet'][mdp.get_common_columns(mice=mice, combined_dict=combined_dict, cortex=True)]
    for k in mice
])

In [ ]:
# set-up for flatmap requirements if required
roi_dict, barcodes_dict, all_bcs_with_source, barcodes_no_source, sample_vol_and_regions = mff.make_dicts_of_mouse_3d_sample_rois(gen_parameters)
proj_top, bf_left_boundaries, bf_right_boundaries = mff.set_up_for_flatmaps(proj_path=gen_parameters['path_to_additional_req'])

Fig. 2a

In [ ]:
normalised_bc_matrices, soma_coordinates_2d = mff.get_source_2d_and_normalised_matrices(roi_dict=roi_dict, gen_parameters=gen_parameters,  proj_top=proj_top, barcodes_no_source=barcodes_no_source)
to_look = mff.make_mean_3d(roi_dict=roi_dict, normalised_bc_matrices=normalised_bc_matrices, gen_parameters=gen_parameters, proj_top=proj_top)
fig, ax = plt.subplots(figsize=(3, 3))
mff.plot_mean_flatmap(to_look=to_look, soma_coordinates_2d=soma_coordinates_2d, fig=fig, ax=ax, bf_left_boundaries=bf_left_boundaries, bf_right_boundaries=bf_right_boundaries)

Fig. 2b-c

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(3, 1))

pltt.plot_all_distance_decay_from_A1(
    fig,
    axes,
    combined_dict=combined_dict,
    all_mice_combined=all_mice_combined,
    mice=mice,
    gen_parameters=gen_parameters,
)

Fig. 2d

In [5]:
mouse_dict_AP_source, mouse_dict_A1_coords = mff.get_AP_coords(
        gen_parameters=gen_parameters
    )
cortical_roi_dict, all_AP_dict, which_soma = mff.find_soma_AP_roi(
    mice=gen_parameters["MICE"],
    proj_path=gen_parameters["proj_path"],
    mouse_dict_AP_source=mouse_dict_AP_source,
    mouse_dict_A1_coords=mouse_dict_A1_coords,
    roi_dict=roi_dict,
)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
mff.plot_AP_soma_cubelet_flatmap(gen_parameters=gen_parameters, fig=fig, ax=ax, roi_dict=roi_dict, bf_left_boundaries=bf_left_boundaries, bf_right_boundaries=bf_right_boundaries, proj_top=proj_top)

Fig. 2e

In [ ]:
fig, ax = plt.subplots(figsize=(1, 1))
AP_position_dict_list_combined, AP_soma_VC_sample = mdp.compute_mean_soma_AP_positions(gen_parameters=gen_parameters)
pltt.plot_ap_vs_visap(ax, AP_soma_VC_sample, gen_parameters)

Fig. 2f

In [ ]:
where_AP_vis, area_AP_dict = mdp.get_area_mean_AP(gen_parameters=gen_parameters, combined_dict=combined_dict, AP_position_dict_list_combined=AP_position_dict_list_combined)
fig, ax = plt.subplots(figsize=(1.4, 1.4))
pltt.plot_area_AP_positions(
    ax=ax,
    fig=fig,
    area_AP_dict=area_AP_dict,
    where_AP_vis=where_AP_vis,
    font_size=gen_parameters["font_size"],
)


Fig. 2g

In [ ]:
areas_to_plot = ['VISam', 'VISrl', 'VISpor', 'VISpl']
fig, axes = plt.subplots(
    nrows=4,          
    ncols=1,        
    figsize=(1.4, 2.8),
    sharex=True       
)
pltt.plot_area_projection_probs(areas_to_plot=areas_to_plot, gen_parameters=gen_parameters, fig=fig, axes=axes, include_dist_as_covariate=True)


Fig. 2h

In [ ]:
fig, ax = plt.subplots(figsize=(0.7, 1.5))  
pltt.plot_vc_projecting_cells(visual_areas=gen_parameters['HVA_cols'], all_mice=all_mice_combined, fig=fig, ax=ax, gen_parameters=gen_parameters)


Fig. 2i-j

In [ ]:
fig = plt.figure(figsize=(15.25/2.54, 4/2.54))
pltt.plot_the_heatmap_and_bubble(gen_parameters=gen_parameters, all_mice_combined=all_mice_combined, fig=fig)


Fig. 2k-m

In [ ]:
cosine_df = mdp.cal_cosine_sim(gen_parameters=gen_parameters, all_mice_combined=all_mice_combined)
to_plot = mdp.perform_motif_analysis(gen_parameters=gen_parameters, barcodes=all_mice_combined)

fig = plt.figure(figsize=(18/2.54, 5.5/2.54))
ax = fig.add_axes([0.1, 0.25, 0.2, 0.6])
cbax = fig.add_axes([0.31, 0.4, 0.01, 0.35])
pltt.add_cosine_sim_plot(ax=ax, cbax=cbax, cosine_df=cosine_df, gen_parameters=gen_parameters)
ax = fig.add_axes([0.46, 0.25, 0.24, 0.6])
pltt.add_motif_volcano_plot(gen_parameters=gen_parameters, to_plot=to_plot, ax=ax)
ax = fig.add_axes([0.83, 0.35, 0.08, 0.5])
pltt.plot_stream_effects(ax=ax, to_plot=to_plot, gen_parameters=gen_parameters)
